# Outline

### Normalize the x, y positions of objects in the .csv of each cell type

1) Extract the height and width of the image from the Image.csv, in order to translate the x to DV axis y to the if AP axis

2) Create a new DF containing the BoundingBox minima and maxima as columns from the Mask_Obj.csv 

3) Add the IMAGE width and height to the mask dataframe by extracting these columns from the Image.csv

4) Determined the anterior, posterior, dorsal and ventral bounds in image space coordinates taking into consideration the orientation of the image

5) Effectively flip the coordinates of identified objects in X and Y by subtracting the Location_Center_X and Location_Center_Y values from the Width and Height, respectively. This will make it so that cells to the right of the image are more anterior and thus have smaller x coordinate values, and cells to the bottom of the image are more ventral and thus have smaller y coordinate values

6) From the Location_Center_X value for each cell, subtract the number of pixels between the anterior edge of the mask and the right side of the image. From the Location_Center_Y value for each cell, subtract the number of pixels between the ventral edge of the mask and the bottom of the image.

7) Normalize the AP and DV axes (scale 0 to 1) by dividing the Location_Center_X value for each cell by the left-to-right diameter(width) of the mask (calculated via the bounding box measurements of the mask present in the Mask_Obj.csv) and the Location_Center_Y values for each cell by the top-to-bottom diameter(height) of the mask.

#### All of these processing steps are accomplished through the normalized_anatomy( ) function, which requires 1 argument (a directory containing at least 3 files: Image.csv, Mask_obj.csv, CT_probes.csv

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme(style = 'white', 
              font = 'arial',
              font_scale = 2)



In [ ]:
def normalized_anatomy(directory):
    
    for a in os.listdir(directory):
        if a.find('normalized') != -1:
            return 'Dataset Already Normalized!'
    """
        This function will normalize the XY pixel coordinates of each cell within an image on a scale from 
        0 to 1 through measurements of the width and height of the image and its mask. The output will be new
        .csv files in the parent directory that contain the suffix "normalized". These new .csv files will 
        have a "CellType" column containing the marker combinations used to identify the celltypes contained within
        that directory. 
        
        Arguments Required (1):
        1. Directory - provide the full filepath (as a string) to the directory containing the output from cellprofiler.
                    This directory must contain the following files:
                    a) ""Image.csv 
                    b) ""Mask_Obj.csv
                    c) ""CT_probes.csv *one file for each probe combination of interest
        
        Output:
        This function will produce a new 'normalized' csv file containing the normalized pixel coordinates for each cell.
        These new csv files can then be concatenated to make a master dataframe of all the cell locations
    
    """
           
    for x in os.listdir(directory):
        if x.find('Image.csv') != -1:
            images = pd.read_csv(directory + x)
        elif x.find('Mask_Obj.csv') != -1:
            masks = pd.read_csv(directory + x)
            
    masks[['ImageHeight', 'ImageWidth']] = images[['Height_RAW', 'Width_RAW']]
    masks['MaskHeight'] = masks['AreaShape_BoundingBoxMaximum_Y'] - masks['AreaShape_BoundingBoxMinimum_Y']
    masks['MaskWidth'] = masks['AreaShape_BoundingBoxMaximum_X'] - masks['AreaShape_BoundingBoxMinimum_X']
    
    ##Assuming the anterior side is the right side of the image
    masks['AnteriorBound'] = masks['ImageWidth'] - masks['AreaShape_BoundingBoxMaximum_X']
    masks['PosteriorBound'] = masks['ImageWidth'] - masks['AreaShape_BoundingBoxMinimum_X']
    
    #Assuming dorsal is to the top of the image
    masks['VentralBound'] = masks['ImageHeight'] - masks['AreaShape_BoundingBoxMaximum_Y']
    masks['DorsalBound'] = masks['ImageHeight'] - masks['AreaShape_BoundingBoxMinimum_Y']
    
    masks['Aspect (H/W)'] = masks['MaskHeight'] / masks['MaskWidth']
    
    for item in os.listdir(directory):
        if (item.find('CT_') != -1) or (item.find('ExpandedMedianNuclei') != -1):
            combo = pd.read_csv(directory + item)
            ##combo = combo[['ImageNumber',
            ##            'ObjectNumber', 
            ##           'Location_Center_X', 
            ##         'Location_Center_Y']]    

            #Assuming right of image is anterior, and top is dorsal
            ##For the center of each cell(object): 
            ##Subtract the Location_Center_X value from the Width of the image
            ##Subtract the Location_Center_Y value from the Height of the image
            AdjustedXlist = []
            AdjustedYlist = []
            for i in range(len(combo)):
                imgnum = combo['ImageNumber'].iloc[i]

                width = images[images['ImageNumber'] == imgnum]['Width_RAW'].iloc[0]
                adj = width - combo['Location_Center_X'].iloc[i]
                AdjustedXlist.append(adj)

                height = images[images['ImageNumber'] == imgnum]['Height_RAW'].iloc[0]
                heightadj = height - combo['Location_Center_Y'].iloc[i]
                AdjustedYlist.append(heightadj)

            combo['AdjustedX'] = AdjustedXlist
            combo['AdjustedY'] = AdjustedYlist

            #Assuming right of image is anterior, and top is dorsal
            ##For the center of each cell(object): 
            ##Subtract the anterior bound value from the AdjustedX valuefor the cell
            ##Subtract the ventral bound value from the AdjustedY value for the cell
            
            baseXlist = []
            baseYlist = []
            for i in range(len(combo)):
                imgnum = combo['ImageNumber'].iloc[i]

                ant_bound = masks[masks['ImageNumber'] == imgnum]['AnteriorBound'].iloc[0]
                vent_bound = masks[masks['ImageNumber'] == imgnum]['VentralBound'].iloc[0]

                baselineX = combo['AdjustedX'].iloc[i] - ant_bound
                baselineY = combo['AdjustedY'].iloc[i] - vent_bound

                baseXlist.append(baselineX)
                baseYlist.append(baselineY)

            combo['BaselineX'] = baseXlist
            combo['BaselineY'] = baseYlist



            ##To normalize the position values from 0 to 1 in the dorsal ventral and anterior posterior planes
            ## Divide the BaselineX and BaselineY values by the width and height of the MASK
            NormalizedXlist = []
            NormalizedYlist = []
            AspectList = []
            for i in range(len(combo)):
                imgnum = combo['ImageNumber'].iloc[i]
                xdiam = (masks[masks['ImageNumber'] == imgnum]['AreaShape_BoundingBoxMaximum_X'].iloc[0]) - (masks[masks['ImageNumber'] == imgnum]['AreaShape_BoundingBoxMinimum_X'].iloc[0])
                ydiam = (masks[masks['ImageNumber'] == imgnum]['AreaShape_BoundingBoxMaximum_Y'].iloc[0]) - (masks[masks['ImageNumber'] == imgnum]['AreaShape_BoundingBoxMinimum_Y'].iloc[0])
                normalX = combo['BaselineX'].iloc[i] / xdiam
                normalY = combo['BaselineY'].iloc[i] / ydiam
                NormalizedXlist.append(normalX)
                NormalizedYlist.append(normalY)
                aspect = masks[masks['ImageNumber'] == imgnum]['Aspect (H/W)'].iloc[0]
                AspectList.append(aspect)
                
            combo['NormalizedX'] = NormalizedXlist
            combo['NormalizedY'] = NormalizedYlist
            combo['Aspect (H/W)'] = AspectList

            celltypelist = []

            ct = item[item.find('CT_'):item.find('.csv')]
            
            if item.find('CT_') == -1:
                ct = 'DAPI'
            
            for i in range(len(combo)):
                celltypelist.append(ct)
            
            combo['CellType'] = celltypelist
            combo.to_csv(directory + item[:item.find('.csv')] + 'normalized.csv')
            
    return 

In [ ]:
def normalized_anatomy_preCropped_NoMask(directory):
    
    for a in os.listdir(directory):
        if a.find('normalized') != -1:
            return 'Dataset Already Normalized!'
    """
        This function will normalize the XY pixel coordinates of each cell within an image on a scale from 
        0 to 1 through measurements of the width and height of the image and its mask. The output will be new
        .csv files in the parent directory that contain the suffix "normalized". These new .csv files will 
        have a "CellType" column containing the marker combinations used to identify the celltypes contained within
        that directory. 
        
        Arguments Required (1):
        1. Directory - provide the full filepath (as a string) to the directory containing the output from cellprofiler.
                    This directory must contain the following files:
                    a) ""Image.csv 
                    b) ""CT_probes.csv *one file for each probe combination of interest
        
        Output:
        This function will produce a new 'normalized' csv file containing the normalized pixel coordinates for each cell.
        These new csv files can then be concatenated to make a master dataframe of all the cell locations
    
    """
           
    for x in os.listdir(directory):
        if x.find('Image.csv') != -1:
            images = pd.read_csv(directory + x)
            
    
    for item in os.listdir(directory):
        if (item.find('CT_') != -1) or (item.find('Neurons') != -1):
            combo = pd.read_csv(directory + item)
           ## combo = combo[['ImageNumber',
           ##              'ObjectNumber', 
           ##              'Location_Center_X', 
           ##              'Location_Center_Y']]

            
            #Assuming right of image is anterior, and top is dorsal
            ##For the center of each cell(object): 
            ##Subtract the Location_Center_X value from the Width of the image
            ##Subtract the Location_Center_Y value from the Height of the image
            
            NormalizedXlist = []
            NormalizedYlist = []
            AspectList = []
            for i in range(len(combo)):
                
                imgnum = combo['ImageNumber'].iloc[i]

                width = images[images['ImageNumber'] == imgnum]['Width_RAW'].iloc[0]
                adj = width - combo['Location_Center_X'].iloc[i]

                height = images[images['ImageNumber'] == imgnum]['Height_RAW'].iloc[0]
                heightadj = height - combo['Location_Center_Y'].iloc[i]

                imgnum = combo['ImageNumber'].iloc[i]
                normalX = adj / width
                normalY = heightadj / height
                
                NormalizedXlist.append(normalX)
                NormalizedYlist.append(normalY)
                
                aspect = height/width
                AspectList.append(aspect)
                
            combo['NormalizedX'] = NormalizedXlist
            combo['NormalizedY'] = NormalizedYlist
            combo['Aspect (H/W)'] = AspectList

            celltypelist = []

            ct = item[item.find('CT_'):item.find('.csv')]
            
            for i in range(len(combo)):
                celltypelist.append(ct)
            
            combo['CellType'] = celltypelist
            combo.to_csv(directory + item[:item.find('.csv')] + 'normalized.csv')
            
    return 


In [ ]:
for item in os.listdir('Z:/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/Spp1_Nos1_Kcng2/CellProfilerOutput/'):
    if item.find('CT_') != -1:
                print(item)

In [ ]:
normalized_anatomy_preCropped_NoMask("Z:/RKDATA/ConfocalData/SDC/RNAscope\/V2/ZI/Sagittal/FINALSET_FIRSTTIER/Spp1_Slc30a3_Pax6/CellProfilerOutput/112922(3mice)/")

## We have two options for making jointplots with the DAPI only in the central relational plot, but not within the KDEs. See below. The first option requires some creativity outside of python

# Slc17a6, Gad2, Pax6------------------------------------------------------------

In [ ]:
SGPdatadir = '/Users/ryankast/Downloads/Slc17a6Gad2Pax6RNAscopeAnalysis/'
normalized_anatomy_preCropped_NoMask(SGPdatadir)

savedir = SGPdatadir + 'JointPlots/'

In [ ]:
normalizedlist = []
for item in os.listdir(SGPdatadir):
    if item.find('normalized') != -1:
        normalizedlist.append(pd.read_csv(SGPdatadir+item))

SGPfulldf = pd.concat(normalizedlist)
SGPfulldf = SGPfulldf.reset_index()

In [ ]:
SGPimage = pd.read_csv(SGPdatadir + 'Slc17a6Gad2Pax6_02252023Image.csv')
SGPimagesdict = dict(zip(SGPimage['ImageNumber'],SGPimage['FileName_RAW'].str[4:29]))
                    

In [ ]:
SGPfulldf['CellType'].value_counts()

In [ ]:
#Use named, discrete colors to create a custom colormap
SGPcm = ['red', 'darkturquoise']

#sns.set(font_scale = 2, style= 'white', palette = None)

#Designate which Cell Types you want to plot
SGPctlist = ['CT_Gad2Neurons',
             'CT_Slc17a6Neurons']

SGPctcmdict = dict(zip(SGPctlist, SGPcm))

In [ ]:
SGPimagesdict

In [ ]:
SGPfulldf[SGPfulldf['ImageNumber'] == 10].iloc[0]['Aspect (H/W)']

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1, len(SGPimage)):

    celltypes = ['CT_Slc17a6Neurons','CT_Gad2Neurons']



    cmap = [SGPctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=SGPfulldf[(SGPfulldf['ImageNumber'] == imgnum) &  
                          (SGPfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 palette = cmap,
                 hue = 'CellType',
                 hue_order= celltypes,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 alpha = 0.8,
                 legend = False,
                 )


    Aspect = SGPfulldf[SGPfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition', fontsize = 24)
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)



    plt.legend(title = 'Cell Type', fontsize = 24,
               labels = ['Gad2+ Neurons',
                         'Slc17a6+ Neurons'], 
               bbox_to_anchor = (2.8, 1.1), 
               edgecolor = 'Black')

    #plt.title(SGPimagesdict[imgnum])
    plt.savefig(savedir + SGPimagesdict[imgnum] + str(celltypes) + '.svg' , bbox_inches = 'tight')


# Pax6,Spp1,Slc30a3 ------------------------------------------------------------

In [ ]:
PSSdatadir = 'Z:/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/Spp1_Slc30a3_Pax6/CellProfilerOutput/112922(3mice)/Marker_ScatterPlots/'
normalized_anatomy_preCropped_NoMask(PSSdatadir)

In [ ]:
savedir = 'Z:/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/PythonAnalysis/Plots/Pax6Spp1Slc30a3/3Mice/JointPlots/'

In [ ]:
normalizedlist = []
for item in os.listdir(PSSdatadir):
    if item.find('normalized') != -1:
        normalizedlist.append(pd.read_csv(PSSdatadir+item))

PSSfulldf = pd.concat(normalizedlist)
PSSfulldf = PSSfulldf.reset_index()

In [ ]:
PSSimage = pd.read_csv(PSSdatadir + 'Slc30a3Spp1Pax6_CellPosePax6_112922Image.csv')
PSSimagesdict = dict(zip(PSSimage['ImageNumber'],PSSimage['FileName_RAW'].str[4:16]))

In [ ]:
#Use named, discrete colors to create a custom colormap
PSScm = ['ghostwhite', 'magenta', 'darkblue', 'darkturquoise',  'silver']

#sns.set(font_scale = 2, style= 'white', palette = None)

#Designate which Cell Types you want to plot
PSSctlist = ['CT_DAPI',
             'CT_Slc30a3_Pax6',
             'CT_Slc30a3_Spp1_Pax6',
             'CT_Spp1_Pax6',
             'CT_Pax6only']

PSSctcmdict = dict(zip(PSSctlist, PSScm))

In [ ]:
PSSctcmdict

In [ ]:
PSSfulldf[PSSfulldf['ImageNumber'] == 10].iloc[0]['Aspect (H/W)']

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1,36):

    celltypes = ['CT_Slc30a3_Spp1_Pax6',
                 'CT_Slc30a3_Pax6',
                 'CT_Spp1_Pax6',
                 'CT_Pax6only']


    cmap = [PSSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=PSSfulldf[(PSSfulldf['ImageNumber'] == imgnum) &  
                          (PSSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 palette = cmap,
                 hue = 'CellType',
                 hue_order= celltypes,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 alpha = 0.8,
                 legend = False)


    Aspect = PSSfulldf[PSSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition', fontsize = 24)
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)



    plt.legend(title = 'Cell Type', fontsize = 24,
               labels = ['Pax6+Only',
                         'Spp1+Pax6+',
                         'Slc30a3+Pax6+',
                         'Slc30a3+Spp1+Pax6+'], 
               bbox_to_anchor = (2.8, 1.1), 
               edgecolor = 'Black')

    #plt.title(PSSimagesdict[imgnum])
    #plt.savefig(savedir + PSSimagesdict[imgnum] + str(celltypes) + '.svg' , bbox_inches = 'tight')


In [ ]:
sns.set_theme(style = 'ticks',
              font = 'helvetica',
              font_scale = 1.5)


plt.figure(figsize = (3,3), dpi =300)
plotdata = PSSfulldf[PSSfulldf['CellType'] == 'CT_Pax6Neurons']

sns.scatterplot(data = plotdata,
            #kind = 'scatter',
            x =  (plotdata['Intensity_IntegratedIntensity_Slc30a3Robust'] / plotdata['AreaShape_Area']),
            y=   (plotdata['Intensity_IntegratedIntensity_Spp1Robust'] / plotdata['AreaShape_Area']),
            s = 20,
            color = 'grey',
            edgecolor = 'grey',
            alpha = 0.1).set(ylim = (-0.02, 1.00), xlim = (-0.02, 1.00))

sns.despine(top = True, 
            right = True)

In [ ]:
sns.scatterplot(data = PSSfulldf,
            #kind = 'scatter',
            x =  plotdata['Intensity_IntegratedIntensity_Slc30a3Robust'] / plotdata['AreaShape_Area'],
            y=   plotdata['Intensity_IntegratedIntensity_Spp1Robust'] / plotdata['AreaShape_Area'],
            s = 50,
            color = 'teal',
            edgecolor = 'white',
            alpha = 0.1).set(ylim = (-0.02, 1), xlim = (-0.02, 1))

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1,22):

    celltypes = ['CT_DAPI']


    cmap = [PSSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=PSSfulldf[(PSSfulldf['ImageNumber'] == imgnum) &  
                          (PSSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 hue = 'CellType',
                 hue_order= celltypes,
                 palette = cmap,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 legend = False,
                 edgecolor = 'Black',
                 linewidth = 0.1)


    Aspect = PSSfulldf[PSSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)

    plt.legend(title = 'Cell Type', 
               labels= ['DAPI'], 

               bbox_to_anchor = (2.8, 1.1), 
               edgecolor = 'Black',
               fontsize = 24,
               title_fontsize=24)



    #plt.title(PSSimagesdict[imgnum])
    #plt.savefig(savedir + PSSimagesdict[imgnum]  + '_DAPI.png' , bbox_inches = 'tight', dpi = 300)

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1,22):

    celltypes = ['CT_Slc30a3_Spp1_Pax6']


    cmap = [PSSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=PSSfulldf[(PSSfulldf['ImageNumber'] == imgnum) &  
                          (PSSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 hue = 'CellType',
                 hue_order= celltypes,
                 palette = cmap,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 legend = False,
                 linewidth = 0.1)


    Aspect = PSSfulldf[PSSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)
    
    plt.legend(title = 'Cell Type', 
           labels= ['Slc30a3+Spp1+Pax6+'], 

           bbox_to_anchor = (2.8, 1.1), 
           edgecolor = 'Black',
           fontsize = 24,
           title_fontsize=24)




    #plt.title(PSSimagesdict[imgnum])
    #plt.savefig(savedir + PSSimagesdict[imgnum]  + '_TriplePosStubborn.png' , bbox_inches = 'tight', dpi = 300)

# Kcng2,Nos1,Spp1------------------------------------------------------------

In [ ]:
cd /Volumes/ryanKast/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/Spp1_Nos1_Kcng2/CellProfilerOutput/Spp1_ii24_Nos1_ii24/

In [ ]:
ls

In [ ]:
KNSdatadir = "./"
normalized_anatomy_preCropped_NoMask(KNSdatadir)


In [ ]:
savedir = '/Volumes/ryanKast/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/PythonAnalysis/Plots/Kcng2Nos1Spp1/JointPlots/'

In [ ]:
KNSdatadir = "./"
normalizedlist = []

for item in os.listdir(KNSdatadir):
    if item.find('normalized') != -1:
        normalizedlist.append(pd.read_csv(KNSdatadir+item))

KNSfulldf = pd.concat(normalizedlist)
KNSfulldf = KNSfulldf.reset_index()

In [ ]:
KNSfulldf['CellType'].unique().tolist()

In [ ]:
#Use named, discrete colors to create a custom colormap
KNScm = ['darkblue', 
         'red', 
         'darkturquoise',
         'silver', 
         'ghostwhite']

#sns.set(font_scale = 2, style= 'white', palette = None)

#Designate which Cell Types you want to plot
KNSctlist = ['CT_Nos1_Spp1_Kcng2', 
             'CT_Nos1_Kcng2', 
             'CT_Spp1_Kcng2', 
             'CT_Kcng2only',
             'CT_DAPI']

KNSctcmdict = dict(zip(KNSctlist, KNScm))

In [ ]:
KNSimages = pd.read_csv('Spp1Nos1Kcng2_CellPoseKcng2_092122Image.csv')

In [ ]:
KNSimages = KNSimages[['ImageNumber', 'FileName_RAW']]

In [ ]:
KNSimages['FileName_RAW'] = KNSimages['FileName_RAW'].str[4:16].tolist()


In [ ]:
KNSimagesdict = dict(zip(KNSimages['ImageNumber'], KNSimages['FileName_RAW']))

In [ ]:
for imgnum in range(1,KNSimages['ImageNumber'].max()):
    
    celltypes = [ 'CT_Nos1_Spp1_Kcng2',  'CT_Spp1_Kcng2', 'CT_Nos1_Kcng2', 'CT_Kcng2only'] #'CT_DAPI']
    
    cmap = [KNSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=KNSfulldf[(KNSfulldf['ImageNumber'] == imgnum) &  
                          (KNSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 palette = cmap,
                 hue = 'CellType',
                 hue_order= celltypes,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 legend = False,
                 )


    Aspect = KNSfulldf[KNSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']

    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)

    plt.legend(title = 'Cell Type', 
               labels= ['Kcng2Only',
                        'Nos1+Kcng2+',
                        'Spp1+Kcng2+',

                        'Nos1+Spp1+Kcng2',
                         ], 

               bbox_to_anchor = (2.8, 1.1), 
               edgecolor = 'Black',
               fontsize = 24,
               title_fontsize=24)

    #plt.title(KNSimagesdict[imgnum])
    plt.savefig(savedir + KNSimagesdict[imgnum] + '.svg' , bbox_inches = 'tight', dpi=300)

In [ ]:
    ##Make one with DAPI and one without iteratively, and then overlay in illustrator
    #fig = plt.figure(figsize=(5,4), dpi =300)
    #ax1 = fig.add_axes([0,0,1,1])

    ##Assign which image in the dataset you want to plot
for imgnum in range(1,28):
        celltypes = [ 'CT_Nos1_Spp1_Kcng2',  'CT_Spp1_Kcng2', 'CT_Nos1_Kcng2', 'CT_Kcng2only'] #'CT_DAPI']

        cmap = [KNSctcmdict[x] for x in celltypes]

        g = sns.jointplot(data=KNSfulldf[(KNSfulldf['ImageNumber'] == imgnum) &  
                              (KNSfulldf['CellType'].isin(celltypes))],
                     x='NormalizedX',
                     y='NormalizedY',
                     palette = cmap,
                     hue = 'CellType',
                     hue_order= celltypes,

                     xlim = (-0.2,1.2),
                     ylim = (-0.2,1.2),
                     s = 80,
                     legend = False,
                     )


        Aspect = KNSfulldf[KNSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']

        g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
        g.fig.set_figwidth(20)
        g.fig.set_figheight(20*Aspect)

        plt.legend(title = 'Cell Type', 
                   labels= ['Kcng2Only',
                            'Nos1+Spp1+Kcng2',
                            'Spp1+Kcng2+',
                            'Nos1+Kcng2+',
                             ], 

                   bbox_to_anchor = (2.8, 1.1), 
                   edgecolor = 'Black',
                   fontsize = 24,
                   title_fontsize=24)

        #plt.title(KNSimagesdict[imgnum])
        #plt.savefig(savedir + KNSimagesdict[imgnum] + '.png' , bbox_inches = 'tight', dpi=300)

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1,28):

    celltypes = ['CT_DAPI']


    cmap = [KNSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=KNSfulldf[(KNSfulldf['ImageNumber'] == imgnum) &  
                          (KNSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 hue = 'CellType',
                 hue_order= celltypes,
                 palette = cmap,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 legend = False,
                 edgecolor = 'Black',
                 linewidth = 0.1)


    Aspect = KNSfulldf[KNSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)

    plt.legend(title = 'Cell Type', 
               labels= ['DAPI'], 

               bbox_to_anchor = (2.8, 1.1), 
               edgecolor = 'Black',
               fontsize = 24,
               title_fontsize=24)



    #plt.title(KNSimagesdict[imgnum])
    plt.savefig(savedir + KNSimagesdict[imgnum]  + '_DAPI.png' , bbox_inches = 'tight', dpi = 300)

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
for imgnum in range(1,28):

    celltypes = ['CT_Nos1_Spp1_Kcng2']


    cmap = [KNSctcmdict[x] for x in celltypes]

    g = sns.jointplot(data=KNSfulldf[(KNSfulldf['ImageNumber'] == imgnum) &  
                          (KNSfulldf['CellType'].isin(celltypes))],
                 x='NormalizedX',
                 y='NormalizedY',
                 hue = 'CellType',
                 hue_order= celltypes,
                 palette = cmap,
                 xlim = (-0.2,1.2),
                 ylim = (-0.2,1.2),
                 s = 80,
                 legend = False,
                 linewidth = 0.1)


    Aspect = KNSfulldf[KNSfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
    g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
    g.fig.set_figwidth(20)
    g.fig.set_figheight(20*Aspect)
    
    plt.legend(title = 'Cell Type', 
           labels= ['Nos1+Spp1+Kcng2'], 

           bbox_to_anchor = (2.8, 1.1), 
           edgecolor = 'Black',
           fontsize = 24,
           title_fontsize=24)




    #plt.title(KNSimagesdict[imgnum])
    #plt.savefig(savedir + KNSimagesdict[imgnum]  + '_TriplePosStubborn.png' , bbox_inches = 'tight', dpi = 300)

# Gad2,Pax6,Kcng2------------------------------------------------------------

In [ ]:
cd /Volumes/ryanKast/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/Gad2_Pax6_Kcng2/CellProfilerOutput/3Mice_110222/

In [ ]:
ls

In [ ]:
GPKdatadir = './'
normalized_anatomy_preCropped_NoMask(GPKdatadir)

In [ ]:
savedir = '/Volumes/ryanKast/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/PythonAnalysis/Plots/Gad2Pax6Kcng2/JointPlots/3Mice/'



In [ ]:
normalizedlist = []
for item in os.listdir(GPKdatadir):
    if item.find('normalized') != -1:
        normalizedlist.append(pd.read_csv(GPKdatadir+item))

GPKfulldf = pd.concat(normalizedlist)
GPKfulldf = GPKfulldf.reset_index()

In [ ]:
GPKfulldf['CellType'].unique().tolist()

In [ ]:
#Use named, discrete colors to create a custom colormap
GPKcm = ['silver', 'darkturquoise', 'magenta', 'darkblue', 'white']

#sns.set(font_scale = 2, style= 'white', palette = None)

#Designate which Cell Types you want to plot
GPKctlist = ['CT_Gad2only', 'CT_Pax6_Gad2', 'CT_Kcng2_Gad2', 'CT_Kcng2_Pax6_Gad2', 'CT_DAPI']

GPKctcmdict = dict(zip(GPKctlist, GPKcm))

In [ ]:
GPKimages = pd.read_csv('Kcng2Pax6Gad2_CellPoseGad2_110222Image.csv')

In [ ]:
GPKimages = GPKimages[['ImageNumber', 'FileName_RAW']]

In [ ]:
GPKimages['FileName_RAW'] = GPKimages['FileName_RAW'].str[4:16].tolist()


In [ ]:
GPKimagesdict = dict(zip(GPKimages['ImageNumber'], GPKimages['FileName_RAW']))

In [ ]:
GPKimagesdict[7]

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
##Assign which image in the dataset you want to plot
for imgnum in range(1,46):

        celltypes =  ['CT_Gad2only',
                     'CT_Kcng2_Gad2',
                     'CT_Kcng2_Pax6_Gad2',
                     'CT_Pax6_Gad2'] #,'CT_DAPI']

        cmap = [GPKctcmdict[x] for x in celltypes]

        g = sns.jointplot(data=GPKfulldf[(GPKfulldf['ImageNumber'] == imgnum) &  
                              (GPKfulldf['CellType'].isin(celltypes))],
                     x='NormalizedX',
                     y='NormalizedY',
                     hue = 'CellType',
                     hue_order= celltypes,
                     palette = cmap,
                     xlim = (-0.2,1.2),
                     ylim = (-0.2,1.2),
                     s = 80,
                     legend = False)


        Aspect = GPKfulldf[GPKfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
        g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
        g.fig.set_figwidth(20)
        g.fig.set_figheight(20*Aspect)


        plt.legend(title = 'Cell Type', 
                   labels = celltypes[-1::-1], 
                   bbox_to_anchor = (2.8, 1.1), 
                   edgecolor = 'Black')
        #plt.title(GPKimagesdict[imgnum])
        plt.savefig(savedir + GPKimagesdict[imgnum]  + '.svg' , bbox_inches = 'tight', transparent = True, dpi = 300)

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
##Assign which image in the dataset you want to plot
for imgnum in range(1,13):

        celltypes =  ['CT_DAPI']

        cmap = [GPKctcmdict[x] for x in celltypes]

        g = sns.jointplot(data=GPKfulldf[(GPKfulldf['ImageNumber'] == imgnum) &  
                              (GPKfulldf['CellType'].isin(celltypes))],
                     x='NormalizedX',
                     y='NormalizedY',
                     hue = 'CellType',
                     hue_order= celltypes,
                     palette = ['lightgray'],
                     xlim = (-0.2,1.2),
                     ylim = (-0.2,1.2),
                     s = 80,
                     legend = False)


        Aspect = GPKfulldf[GPKfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
        g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
        g.fig.set_figwidth(20)
        g.fig.set_figheight(20*Aspect)


        plt.legend(title = 'Cell Type', 
                   labels = celltypes[-1::-1], 
                   bbox_to_anchor = (2.8, 1.1), 
                   edgecolor = 'Black')
        #plt.title(GPKimagesdict[imgnum])
        plt.savefig(GPKimagesdict[imgnum] + '_DAPI.svg' , bbox_inches = 'tight', transparent = True, dpi = 300)

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
##Assign which image in the dataset you want to plot
for imgnum in range(1,13):

        celltypes =  ['CT_Kcng2_Pax6_Gad2']

        cmap = [GPKctcmdict[x] for x in celltypes]

        g = sns.jointplot(data=GPKfulldf[(GPKfulldf['ImageNumber'] == imgnum) &  
                              (GPKfulldf['CellType'].isin(celltypes))],
                     x='NormalizedX',
                     y='NormalizedY',
                     hue = 'CellType',
                     hue_order= celltypes,
                     palette = cmap,
                     xlim = (-0.2,1.2),
                     ylim = (-0.2,1.2),
                     s = 80,
                     legend = False)


        Aspect = GPKfulldf[GPKfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
        g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
        g.fig.set_figwidth(20)
        g.fig.set_figheight(20*Aspect)


        plt.legend(title = 'Cell Type', 
                   labels = celltypes[-1::-1], 
                   bbox_to_anchor = (2.8, 1.1), 
                   edgecolor = 'Black')
        #plt.title(GPKimagesdict[imgnum])
        plt.savefig(savedir + GPKimagesdict[imgnum] + '_TripPosStubborn.png' , bbox_inches = 'tight',transparent = True, dpi = 300)

# Pax6, Pmfbp1, Tacr3------------------------------------------------------------

In [ ]:
cd /Volumes/ryanKast/RKDATA/ConfocalData/SDC/RNAscope/V2/ZI/Sagittal/FINALSET_FIRSTTIER/

In [ ]:
cd Pax6_Tacr3_Pmfbp1/

In [ ]:
PTPdatadir = 'Pax6_Tacr3_Pmfbp1/'
normalizedlist = []
for item in os.listdir(PTPdatadir):
    if item.find('normalized') != -1:
        normalizedlist.append(pd.read_csv(PTPdatadir+item))

PTPfulldf = pd.concat(normalizedlist)
PTPfulldf = PTPfulldf.reset_index()

In [ ]:
PTPfulldf['CellType'].value_counts()

In [ ]:
#Use named, discrete colors to create a custom colormap
PTPcm = ['red', 'limegreen', 'darkviolet', 'darkorange', 'grey']

#sns.set(font_scale = 2, style= 'white', palette = None)

#Designate which Cell Types you want to plot
PTPctlist = ['CT_Total_Gad2', 'CT_Pax6_Gad2', 'CT_Kcng2_Gad2', 'CT_Kcng2_Pax6_Gad2', 'DAPI']

PTPctcmdict = dict(zip(PTPctlist, PTPcm))

In [ ]:
PTPimages = pd.read_csv('all_robustImage.csv')

In [ ]:
imagesdict[imgnum]

In [ ]:
GPKimages = GPKimages[['ImageNumber', 'FileName_RAW']]

In [ ]:
GPKimages['FileName_RAW'] = GPKimages['FileName_RAW'].str[4:16].tolist()


In [ ]:
GPKimagesdict = dict(zip(GPKimages['ImageNumber'], GPKimages['FileName_RAW']))

In [ ]:
GPKimagesdict[7]

In [ ]:
##Make one with DAPI and one without iteratively, and then overlay in illustrator
#fig = plt.figure(figsize=(5,4), dpi =300)
#ax1 = fig.add_axes([0,0,1,1])

##Assign which image in the dataset you want to plot
imgnum = 8

celltypes = ['CT_Total_Gad2']


cmap = [GPKctcmdict[x] for x in celltypes]

g = sns.jointplot(data=GPKfulldf[(GPKfulldf['ImageNumber'] == imgnum) &  
                      (GPKfulldf['CellType'].isin(celltypes))],
             x='NormalizedX',
             y='NormalizedY',
             hue = 'CellType',
             hue_order= celltypes,
             palette = cmap,
             xlim = (-0.2,1.2),
             ylim = (-0.2,1.2),
             s = 80,
             legend = False)


Aspect = GPKfulldf[GPKfulldf['ImageNumber'] == imgnum].iloc[0]['Aspect (H/W)']
g.set_axis_labels('Normalized Anterior-Posterior Postition', 'Normalized Dorsal-Ventral Postition')
g.fig.set_figwidth(20)
g.fig.set_figheight(20*Aspect)


plt.legend(title = 'Cell Type', 
           labels = celltypes[-1::-1], 
           bbox_to_anchor = (2.8, 1.1), 
           edgecolor = 'Black')
#plt.title(GPKimagesdict[imgnum])
plt.savefig(GPKimagesdict[imgnum] + str(celltypes) + '.png' , bbox_inches = 'tight')

# Other Exploratory Options

In [ ]:
## Make the jointplot with DAPI in white (DAPI also needs to be first CT in the column) and 
## and include an edgecolor assignment, 
## and blow up the height to make the desired KDEs large enough


cm = ['red', 'deepskyblue','gold', 'white']

celltypes = ['CT_Kcng2_Nos1', 'CT_Kcng2_Spp1', 'CT_Kcng2_Nos1_Spp1','DAPI']

sns.jointplot(data=fulldf[(fulldf['ImageNumber'] == 10) &  
                      (fulldf['CellType'].isin(celltypes))].reindex(index=fulldf[(fulldf['ImageNumber'] == 10) &  
                      (fulldf['CellType'].isin(celltypes))].index[::-1]),
             x='NormalizedX',
             y='NormalizedY',
             hue = 'CellType',
             hue_order= celltypes,
             palette = cm[:len(celltypes)],
             ratio=1,
             height = 15,
             space = 0.05,
             edgecolor='grey')

#handles, labels = g.ax_joint.get_legend_handles_labels()


In [ ]:
cm = ['red', 'deepskyblue','gold', 'lightgrey' ]

celltypes = ['CT_Kcng2_Nos1', 'CT_Kcng2_Spp1', 'CT_Kcng2_Nos1_Spp1',   'DAPI']


sns.scatterplot(data=fulldf[(fulldf['ImageNumber'] == 4) &  
                      (fulldf['CellType'].isin(celltypes))],
             x='NormalizedX',
             y='NormalizedY',
             hue = 'CellType',
             hue_order= celltypes,
             palette = cm[:len(celltypes)],
             alpha = 0.5)


In [ ]:
penguins = sns.load_dataset('penguins')

In [ ]:
fulldf

In [ ]:
sns.FacetGrid()

In [ ]:
cm = ['Red', 'deepskyblue','gold', 'lightgrey' ]

celltypes = ['CT_Kcng2_Nos1', 'CT_Kcng2_Spp1', 'CT_Kcng2_Nos1_Spp1',  'DAPI']


df2 = fulldf[fulldf['CellType'] == 'DAPI']

graph = sns.FacetGrid(data=fulldf[(fulldf['ImageNumber'] == 3) &  
                      (fulldf['CellType'].isin(celltypes[:-1]))])
                        
                        #hue = 'CellType',
                        #hue_order= celltypes[:-1])
            
graph.map(sns.scatterplot(data=fulldf[(fulldf['ImageNumber'] == 3) &  
                      (fulldf['CellType'].isin(celltypes[:-1]))],
                       palette = cm[:len(celltypes[:-1])],
                       #alpha = 0.8,
                       hue = 'CellType',
                       hue_order = celltypes[:-1],
                       x='NormalizedX',
                       y='NormalizedY'))
          
graph.x = df2[df2['ImageNumber'] == 3].NormalizedX
graph.y = df2[df2['ImageNumber'] == 3].NormalizedY

graph.plot_joint(plt.scatter, c='lightgrey', s=50)

In [ ]:
df2

In [ ]:

pal = ['darkorange', 'darkviolet', 'limegreen', 'red']

g = sns.FacetGrid(fulldf[(fulldf['ImageNumber'] == 3) & (fulldf['CellType'].isin(['CT_Kcng2_Pax6_Gad2', 
                                                    'CT_Kcng2_Gad2', 
                                                    'CT_Pax6_Gad2',
                                                     'CT_Total_Gad2']))], 
                  row="CellType", 
                  hue="CellType",
                  hue_order= ['CT_Kcng2_Pax6_Gad2', 'CT_Kcng2_Gad2','CT_Pax6_Gad2','CT_Total_Gad2'],
                  row_order= ['CT_Kcng2_Pax6_Gad2', 'CT_Kcng2_Gad2','CT_Pax6_Gad2','CT_Total_Gad2'],
                  aspect=5, 
                  height=2,
                  palette = pal,
                  )

# Draw the densities in a few steps
g.map(sns.kdeplot, "NormalizedY", bw_adjust= 1, clip_on=False, fill=True, alpha=1, linewidth=1)

g.map(sns.kdeplot, "NormalizedY", clip_on=True, color="w", lw=1, bw_adjust=1)
g.map(plt.axhline, y=0, lw=2, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(-0.2, .3, label, fontsize = 14, fontweight="bold", color=color, ha="left", va="center", transform=ax.transAxes)


g.map(label, "NormalizedX")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=1)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)


In [ ]:
image = pd.read_csv('RNAscope_v1_RB_RB_O3Image.csv')

imagepercentDF = pd.DataFrame(columns =[x for x in image.columns.tolist() if 
                                          (x.find('Count_') != -1) & 
                                          (x.find('Nuclei') == -1) &
                                          (x.find('Mask') == -1)])

for col in imagepercentDF.columns.tolist():
    imagepercentDF[col] = image[col]

for item in CT_columns:
    if item.find('Total') == -1:
        for value in CT_columns:
            
            imagepercentDF['Percent: ' + item + ' / ' + value] = image[item] / image[value]

imagepercentDF.insert(0, 'FileName_RAW', image['FileName_RAW'])
imagepercentDF.insert(0, 'ImageNumber', image['ImageNumber'])
imagepercentDF.info()

In [ ]:
mouse = image.sum()


mousepercentDF = pd.Series(index =[x for x in mouse.index.tolist() if 
                                          (x.find('Count_') != -1) & 
                                          (x.find('Nuclei') == -1) &
                                          (x.find('Mask') == -1)],dtype = 'float64')

for col in mousepercentDF.index.tolist():
    mousepercentDF[col] = mouse[col]

for item in CT_columns:
    if item.find('Total') == -1:
        for value in CT_columns:
            mousepercentDF['Percent: ' + item + ' / ' + value] = mouse[item] / mouse[value]


mousepercentDF

In [ ]:
#cm = ['darkorange','limegreen', 'darkviolet']
sns.(data=fulldf, # &  (fulldf['CellType'].isin())], 
             x_vars='NormalizedX',
             y_vars='NormalizedY',
             hue = 'CellType',
             #alpha = 0.5,
             #palette = cm
             )

In [ ]:
mousepercentDF['Gad2+ (SinglePositive)'] = 1 - (mousepercentDF['Percent: Count_Kcng2_Gad2 / Count_Total_Gad2'] +
                                            mousepercentDF['Percent: Count_Pax6_Gad2 / Count_Total_Gad2'])


In [ ]:
mousepercentDF['Gad2+ (SinglePositive)'] = mousepercentDF['Count_Total_Gad2'] - (mousepercentDF['Count_Kcng2_Gad2'] + mousepercentDF['Count_Pax6_Gad2'])

In [ ]:
plt.figure(figsize = (5,5), dpi =300)
cm = ['darkviolet', 'darkorange','limegreen', 'lightgray']
labelz = ['Kcng2+Gad2+/Gad2+', 'Kcng2+Pax6+Gad2+/Gad2+', 'Pax6+Gad2+/Gad2+', 'Gad2+ (SinglePositive)']
piedata = mousepercentDF.loc[mousepercentDF.index.isin(['Percent: Count_Kcng2_Gad2 / Count_Total_Gad2', 
                                   'Percent: Count_Pax6_Gad2 / Count_Total_Gad2', 
                                   'Percent: Count_Kcng2_Pax6_Gad2 / Count_Total_Gad2',
                                    'Gad2+ (SinglePositive)'])]
#explode = (0.1,0.1,0.1)
plt.pie(piedata,
         colors = cm,
         startangle=90,
         autopct = '%1.1f%%')

plt.legend(bbox_to_anchor = [1,0.6],labels=labelz)
plt.title('Relative Sizes of\nDistinct Molecular Subtypes\nof Gad2+ ZI Neurons (MouseID = 7200-2)',)                              
plt.show()                             

In [ ]:
plt.figure(figsize = (5,5), dpi =300)
cm = ['darkviolet', 'darkorange','limegreen', 'lightgray']
labelz = ['Kcng2+Gad2+ / Gad2+', 'Kcng2+Pax6+Gad2+ / Gad2+', 'Pax6+Gad2+ / Gad2+']
piedata = mousepercentDF.loc[mousepercentDF.index.isin(['Percent: Count_Kcng2_Gad2 / Count_Total_Gad2', 
                                   'Percent: Count_Pax6_Gad2 / Count_Total_Gad2', 
                                   'Percent: Count_Kcng2_Pax6_Gad2 / Count_Total_Gad2',
                                    ])]
#explode = (0.1,0.1,0.1)
plt.pie(piedata,
         #labels = labelz,
         colors = cm,
         startangle=90,
         autopct = '%1.1f%%')
                                
plt.legend(bbox_to_anchor = [1,0.6],labels=labelz)
plt.show()                             

In [ ]:
mousepercentDF.index.isin(['Percent: Count_Kcng2_Gad2 / Count_Total_Gad2', 'Percent: Count_Pax6_Gad2 / Count_Total_Gad2', 'Percent: Count_Kcng2_Pax6_Gad2 / Count_Total_Gad2'])

In [ ]:
mousepercentDF.index

In [ ]:
plt.figure(figsize = (5,5), dpi =300)
cm = ['darkviolet', 'darkorange','limegreen', 'lightgray']
labelz = ['Kcng2+Gad2+/Gad2+', 'Kcng2+Pax6+Gad2+/Gad2+', 'Pax6+Gad2+/Gad2+', 'Gad2+ (SinglePositive)']
piedata = mousepercentDF.loc[mousepercentDF.index.isin(['Count_Kcng2_Gad2', 
                                   'Count_Pax6_Gad2', 
                                   'Count_Kcng2_Pax6_Gad2',
                                    'Gad2+ (SinglePositive)'])]
#explode = (0.1,0.1,0.1)
plt.pie(piedata,
         colors = cm,
         startangle=90,
         autopct = '%1.1f%%')

plt.legend(bbox_to_anchor = [1,0.6],labels=labelz)
plt.title('Relative Sizes of\nDistinct Molecular Subtypes\nof Gad2+ ZI Neurons (Mouse ID = 7200-2)',)                              
plt.show()                             